In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.regularizers import L1
from tensorflow.keras import backend as K
import keras
from keras import ops
from keras import layers

2025-04-22 14:15:41.061713: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-22 14:15:41.848408: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%cd ../original_data

/home/binhducvu/tree_kernels/decoder_sandbox/original_data


In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator()

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

class VEncoder(keras.Model):
    def __init__(self, input_dim: int, latent_dim:int, layer_1:int,
                 layer_2:int, **kwargs):
        super().__init__(**kwargs)

        encoder_inputs = keras.Input(shape=(input_dim,))
        x = layers.Dense(layer_1, activation = "relu")(encoder_inputs)
        x = layers.Dense(layer_2, activation = "relu")(x)
        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        self.encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    def call(self, x):
        return self.encoder(x)

class VDecoder(keras.Model):
    def __init__(self, latent_dim:int, output_dim:int, layer_1:int,
                 layer_2:int, **kwargs):
        super().__init__(**kwargs)

        latent_inputs = keras.Input(shape=(latent_dim,))
        x = layers.Dense(layer_2, activation = "relu")(latent_inputs)
        x = layers.Dense(layer_1, activation = "relu")(x)
        decoder_outputs = layers.Dense(output_dim, activation="sigmoid")(x)
        
        self.decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    def call(self, x):
        return self.decoder(x)


class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
    
    def call(self, x):
        z_mean, z_log_var, z = self.encoder(x)
        reconstruction = self.decoder(z)
        return z_mean, z_log_var, reconstruction

    # also used during validation
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def calculate_reconstruction_loss(self, data, reconstruction):
        """
        In case of computer vision tasks use the following:
            keras.losses.binary_crossentropy(data, reconstruction),
            axis=(1, 2),
        """
        return ops.mean(ops.sum(keras.losses.binary_crossentropy(data, reconstruction)))
    
    def calculate_kl_loss(self, z_mean, z_log_var):
        kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
        kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
        return kl_loss
    
    def calculate_total_loss(self, reconstruction_loss, kl_loss):
        return reconstruction_loss + kl_loss 
        
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            
            reconstruction_loss = self.calculate_reconstruction_loss(data, reconstruction)
            kl_loss = self.calculate_kl_loss(z_mean, z_log_var)
            total_loss = self.calculate_total_loss(reconstruction_loss, kl_loss)

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data, training=False)
        reconstruction = self.decoder(z)
        
        reconstruction_loss = self.calculate_reconstruction_loss(data, reconstruction)
        kl_loss = self.calculate_kl_loss(z_mean, z_log_var)
        total_loss = self.calculate_total_loss(reconstruction_loss, kl_loss)
        
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [6]:
def vae(dat, latent_rate=0.2, runs=10, drop_index = True):
    # read data, drop first col if needed, split into train/val/test
    if drop_index == True:
        trn = pd.read_csv(dat + '_trn.csv').iloc[:, 1:]
        tst = pd.read_csv(dat + '_tst.csv').iloc[:, 1:]
    else:
        trn = pd.read_csv(dat + '_trn.csv')
        tst = pd.read_csv(dat + '_tst.csv')
        
    trn = trn[trn['y']==1].drop('y', axis=1)
    trn, val = train_test_split(trn, test_size = 0.1)
    trn = trn.reset_index().drop('index', axis=1)
    val = val.reset_index().drop('index', axis=1)
    
    latent_dim = round(latent_rate * trn.shape[1])
    # encoding and minmax scaling, fit on train, transform on rest
    enc = OneHotEncoder(sparse_output=False)
    enc_cols = trn.select_dtypes('object')
    encoded = pd.DataFrame(enc.fit_transform(enc_cols), 
                          columns = enc.get_feature_names_out())
    trn.drop(enc_cols.columns, axis=1, inplace=True)
    trn = pd.concat([trn, encoded], axis=1)
    
    encoded=pd.DataFrame(enc.transform(tst.select_dtypes('object')), 
                        columns = enc.get_feature_names_out())
    tst.drop(enc_cols.columns, axis=1, inplace=True)
    tst = pd.concat([tst, encoded], axis=1)
    
    encoded=pd.DataFrame(enc.transform(val.select_dtypes('object')),
                        columns = enc.get_feature_names_out())
    val.drop(enc_cols.columns, axis=1, inplace=True)
    val  = pd.concat([val, encoded], axis=1)
    cols = trn.columns
    
    scaler = MinMaxScaler()
    trn = scaler.fit_transform(trn)
    tst = scaler.transform(tst)
    val = scaler.transform(val)
    
    # define autoencoder
    input_dim = trn.shape[1]   
    first_dim = round(input_dim - (input_dim - latent_dim) * 1/3)
    second_dim = round(input_dim - (input_dim - latent_dim) * 2/3)

    for i in range(runs):
        encoder = VEncoder(input_dim, latent_dim, first_dim, second_dim)
        decoder = VDecoder(latent_dim, input_dim, first_dim, second_dim)
        vautoencoder = VAE(encoder, decoder)
        vautoencoder.compile(optimizer='adam')
        history = vautoencoder.fit(trn,
                          epochs = 50, 
                          validation_data = val,
                          shuffle = True,
                          verbose=0)
        vae = vautoencoder.predict(tst)[2]
        vae = pd.DataFrame(scaler.inverse_transform(vae), columns = cols)
        if enc.get_feature_names_out().size:
            enc_cols = enc.inverse_transform(vae[enc.get_feature_names_out()])
            vae = vae.drop(enc.get_feature_names_out(), axis=1)
            vae[enc.feature_names_in_] = enc_cols
        vae.to_csv("../vae_data/" + dat + '/' + str(latent_rate) + '_run' + str(i+1) +  ".csv", index=False)
    print("Finished " + dat)
    return None

In [19]:
compressions = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

In [20]:
for compression in compressions:
    vae('abalone', latent_rate = compression)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Finished abalone
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Finished abalone
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [ ]:
for compression in compressions:
    vae('adult', compression, drop_index=False)

In [21]:
for compression in compressions:
    vae('banknote', compression)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Finished banknote
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Finished banknote
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st

In [22]:
for compression in compressions:
    vae('bc', compression)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Finished bc
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Finished bc
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0

In [23]:
for compression in compressions:
    vae('car', compression)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Finished car
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Finished car
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

In [24]:
for compression in compressions:
    vae('churn', compression)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Finished churn
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Finished churn
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/

In [25]:
for compression in compressions:
    vae('credit', compression)

105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Finished credit
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step
Finished credit
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step
105/105 ━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
for compression in compressions:
    vae('diabetes', compression)

In [ ]:
for compression in compressions:
    vae('dry_bean', compression)

In [ ]:
for compression in compressions:
    vae('forestfires', compression)

In [ ]:
for compression in compressions:
    vae('hd', compression)

In [ ]:
for compression in compressions:
    vae('king', compression)

In [ ]:
for compression in compressions:
    vae('marketing', compression)

In [ ]:
for compression in compressions:
    vae('mushroom', compression)

In [ ]:
for compression in compressions:
    vae('obesity', compression)

In [ ]:
for compression in compressions:
    vae('plpn', compression)

In [ ]:
for compression in compressions:
    vae('sick', compression)

In [ ]:
for compression in compressions:
    vae('soybean', compression)

In [ ]:
for compression in compressions:
    vae('spambase', compression)

In [ ]:
for compression in compressions:
    vae('student', compression)

In [ ]:
for compression in compressions:
    vae('wq', compression)